# Text Vectorization

## Setup

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import random
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation
import dill
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.stem.snowball import SnowballStemmer
from nltk.stem.wordnet import WordNetLemmatizer
try:
  from unidecode import unidecode
except:
  !pip install unidecode
  from unidecode import unidecode

pd.set_option('display.max_rows', None)  ###
pd.set_option('display.max_columns', None)  ###
pd.set_option('display.width', None)  ###
pd.set_option('display.max_colwidth', None)  ###

import warnings
warnings.filterwarnings('ignore')

import spacy
try:
    nlp = spacy.load('en_core_web_sm', disable = ['parser','ner'])
except:
    from spacy.cli import download

    download("en_core_web_sm")
    nlp = spacy.load('en_core_web_sm', disable = ['parser','ner'])

In [ ]:
try:
  from google.colab import drive
  IN_COLAB=True
except:
  IN_COLAB=False

if IN_COLAB:
  print("We're running Colab")

if IN_COLAB:  
  # Mount the Google Drive at mount
  mount='/content/gdrive'
  print("Colab: mounting Google drive on ", mount)
  # connect your colab with the drive
  drive.mount(mount)

 # Switch to the directory on the Google Drive that you want to use
  import os
  path_to_repo = mount + "/MyDrive/MIMIC-III Text Mining/LOS_FINAL/"

else:
  # Setup Repository
  with open("repo_info.txt", "r") as repo_info:
      path_to_repo = repo_info.readline()

  
print(path_to_repo)

path_to_data = f"{path_to_repo}data/"
path_to_raw = f"{path_to_data}raw/"
path_to_processed = f"{path_to_data}processed/"
path_to_lda = f"{path_to_data}lda/"
path_to_icd = f"{path_to_data}icd_codes/"
path_to_models = f"{path_to_repo}models/"
path_to_results = f"{path_to_repo}results/"

In [ ]:
# PARAMETERS

session_seed = 42 # set seed for our session
test_proportion = 0.2

MAX_FEATURES = 10000 # maximum number of features
min_df = 5 # minimum frequency
max_df = 0.8 # maximum frequency
N_GRAM = (1,2) # n_gram range

seed_tag = f'_{session_seed}'

random.seed(session_seed)

## Process the dataset

In [ ]:
# import dataset
file = f'{path_to_processed}df_mixed_discharge.csv.gzip'
raw_df = pd.read_csv(file, compression = 'gzip', low_memory=False)

In [ ]:
# examine the variables (names)
raw_df.columns

In [ ]:
# drop the variables to be exempted from the analysis and rename new dataset
df = raw_df.drop(columns = ['hadm_id', 'subject_id','icu_los'])


In [ ]:
# Transform the LOS timedelta to days (as decimal values to increase precision)
df['los'] = pd.to_timedelta(df.los)
df['los'] = df.los/pd.to_timedelta(1, unit='D')

In [ ]:
# selection criterion : only patients 18 and older and with a length of stay or 1 day or greater
df = df.loc[(df['age']>=18) & (df['los']>=1),:]
# compare size of dataframe before and after selection
len(raw_df), len(df)

In [ ]:
# check proportion of missing values
missing = pd.DataFrame(df.isna().mean(), columns = ['proportions'])
missing.sort_values('proportions',ascending = False)

In [ ]:
# drop variables having more than 20 % missing values
print(f"Variables with more than 20% Missing Values: {list(missing.loc[missing.proportions >= 0.2].index)}")
df = df.drop(columns=missing.loc[missing.proportions >= 0.2].index)

In [ ]:
# impute missing values
df = df.interpolate()
df.isna().mean()

## Binarize LOS

In [ ]:
# check distribution of outcome
sns.catplot(kind='box', data=df, y='los');
sns.displot(data=df, x="los", bins=30, kde=True) ;

In [ ]:
# compute Lower and Upper Fence according to Tukey's criteria
y = df['los']
Q1 = np.percentile(y, 25)
Q3 = np.percentile(y, 75)
IQR = Q3-Q1
LF = Q1 - 1.5*IQR
UF = Q3 + 1.5*IQR
print(f'First quartile = {Q1:.3f}, Third Quartile = {Q3:.3f}, Interquartile Interval = {IQR:.3f}')
print(f'Lower Fence = {LF:.3f}, Upper Fence = {UF:.3f}')

In [ ]:
# create categorical LOS variable where prolonged LOS is any value greater than Upper Fence
df['los_cat'] = df['los']> UF
df = df.drop(columns=['los'])

## Heavier Preprocessing & BoW - Define Functions

In [ ]:
stemmer = SnowballStemmer("english", ignore_stopwords=True) # we initialize our stemmer
stop_words = stopwords.words('english') # nltk stopwords
lemmatizer = WordNetLemmatizer() # lemmatizer in case we want to lemmatize instead of stemming

In [ ]:
def remove_stopwords(text):
    text =  " ".join([x for x in text.split()if x not in stop_words]) # delete stopwords from text
    return text

def heading_clean(text):
  text = re.sub(r'\[\*\*(.+?)\*\*\]', "", text) # we take out information in brackets
  text = re.sub(r'(Admission Date:)|(Discharge Date:)|(Date of Birth:)|(Name:)|(Unit No:)', "", text, flags = re.I)
  if preproc_heavier:
    text = re.sub(r'Sex:', "", text, flags = re.I)
    text = text.lower() # lowercase text
    text = unidecode((text))
    text = re.sub(r'[^\w\s]','',text)
    if lemmatize == True:
        if use_spacy == True:
            doc = nlp(text)
            text = " ".join([token.lemma_ for token in doc])
        else:
            text =  " ".join([lemmatizer.lemmatize(x) for x in text.split()])
    else:
        text =  " ".join([stemmer.stem(x) for x in text.split()])
    text = remove_stopwords(text)
  return text

In [ ]:
def vectorize_to_dataframe(df, vectorizer_obj):
    """
    Function to return a dataframe from our vectorizer results
    """
    df = pd.DataFrame(data = df.toarray(), columns = vectorizer_obj.get_feature_names())
    return df

In [ ]:
def vectorize_features(X_train, X_test, method = 'frequency'):
    """
    Function to perform vectorization of our test sets
    X_train, X_test, X_val: our dataframes
    method: either 'frequency', 'tf_idf', 'onehot' to employ a different BoW technique
    """
    # initialize our vectorizer
    if method == 'tf_idf':
        vectorizer = TfidfVectorizer(ngram_range=N_GRAM, min_df=min_df, max_df=max_df, max_features=MAX_FEATURES)
    elif method == 'frequency':
        vectorizer = CountVectorizer(ngram_range=N_GRAM, min_df=min_df, max_df=max_df, max_features=MAX_FEATURES)
    elif method == 'onehot':
        vectorizer = CountVectorizer(ngram_range=N_GRAM, min_df=min_df, max_df=max_df, max_features=MAX_FEATURES, binary = True)
        
    train_text = vectorizer.fit_transform(X_train.text)
    train_text = vectorize_to_dataframe(train_text, vectorizer)
    test_text = vectorizer.transform(X_test.text)
    test_text = vectorize_to_dataframe(test_text, vectorizer)
    return train_text, test_text

In [ ]:
def save_dataframes(train, test, method, target = False):
    """
    Function to save our dataframes
    train: train set to be saved
    method: method through which we have processed the dataframes, needed as save keyword
    test: test set to be saved
    """
    # need to reset the index
    train.reset_index(inplace=True, drop = True)
    # save our dataset up to now in feather format
    train.to_feather('{}{}train_{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag))
    # need to reset the index
    test.reset_index(inplace=True, drop = True)
    # save our dataset up to now in feather format
    test.to_feather('{}{}test_{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag))

In [ ]:
def load_datasets(method):
    """
    Function to load train, test and validation set based on the chosen method
    method: string for the processing method we want to load
    """
    global path_to_processed
    # load it back
    train = pd.read_feather('{}{}train_{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag))
    test = pd.read_feather('{}{}test_{}{}{}'.format(path_to_processed, method, seed_tag, preproc_tag, lemma_tag))
    return train, test

### Vectorization Loop

In [ ]:
# Iterate over our main methods of vectorization

vect_dict = {'stemming': (False, False),
             'spacy': (True, True)}

             
for key, value in vect_dict.items():
    print(key)
    # PARAMETERS

    lemmatize = value[0] # set to false if we want to do stemming
    lemma_tag = str(np.where(lemmatize, "_lemma",""))
    use_spacy = value[1]
    if use_spacy: lemma_tag = str(np.where(lemmatize, "_lemma_spacy",""))

    preprocessing = True # set to true if we want to clean and perform some preprocessing
    preproc_heavier = True # set to True if we want a heavier preprocessing
    preproc_tag_2 = np.where(preproc_heavier, '_heavier', '')
    preproc_tag = np.where(preprocessing, f'_preproc{preproc_tag_2}', f'{preproc_tag_2}')
  
    if preprocessing:
        try:
            df = pd.read_feather(f'{path_to_processed}df_los{preproc_tag}{lemma_tag}')
            print('Dataframe Loaded')
        except:
            df['text'] = df.text.apply(lambda x: heading_clean(x))
            # need to reset the index
            df.reset_index(inplace=True, drop = True)
            # save our dataset up to now in feather format
            df.to_feather(f'{path_to_processed}df_los{preproc_tag}{lemma_tag}')
            print('Dataframe Saved')
        # split the data into training and test
        train, test = train_test_split(df, train_size=0.80, stratify = df['los_cat'], random_state=42)
    # perform vectorization
    method_list = ['frequency', 'onehot','tf_idf']
    train_clean = train.drop(columns = 'text')
    train_clean.reset_index(inplace = True, drop = True)
    test_clean = test.drop(columns = 'text')
    test_clean.reset_index(inplace = True, drop = True)
    for method in method_list:
        print(method)
        # for each method we perform vectorization
        train_text, test_text = vectorize_features(train, test, method = method)
        # Drop the text column and concatenate it
        X_train = pd.concat([train_clean, train_text], axis = 1)
        X_test = pd.concat([test_clean, test_text], axis = 1)
        assert X_train.shape[0] == train_clean.shape[0]
        assert X_test.shape[0] == test_clean.shape[0]
        # and save the dataframes
        save_dataframes(X_train, X_test, method = method)
        # And also perform LDA
        try:
            with open(f'{path_to_lda}lda{seed_tag}{method}{preproc_tag}{lemma_tag}', 'rb') as file: # and save the fitted model
                lda = dill.load(file)
            train_lda = lda.transform(train_text)
        except:
            lda = LatentDirichletAllocation(n_components = 300, random_state = session_seed, n_jobs = -1)
            train_lda = lda.fit_transform(train_text)
        train_lda = pd.DataFrame(train_lda)
        train_lda.columns=["F"+str(i) for i in range(0, len(train_lda.columns))]
        test_lda = lda.transform(test_text)
        test_lda = pd.DataFrame(test_lda)
        test_lda.columns=["F"+str(i) for i in range(0, len(test_lda.columns))]
        # Drop the text column and concatenate it
        X_train = pd.concat([train_clean, train_lda], axis = 1)
        X_test = pd.concat([test_clean, test_lda], axis = 1)
        assert X_train.shape[0] == train_clean.shape[0]
        assert X_test.shape[0] == test_clean.shape[0]
        save_dataframes(X_train, X_test, method = f'lda_{method}')
        with open(f'{path_to_lda}lda{seed_tag}{method}{preproc_tag}{lemma_tag}', 'wb') as file: # and save the fitted model
            dill.dump(lda, file)